# EXTRACCIÓN DE PARTES METAR

## En este Notebook se va a desarrollar el proceso de estracción de todos los partes METAR del Aeropuerto Adolfo Suárez Madrid-Barajas en el periodo comprendido entre el 01-11-2022 y el 31-10-2023, es decir, 1 año completo.

Un parte o código METAR es el estándar internacional en aviación del formato del código utilizado para emitir periódicamente informes de las observaciones meteorológicas en los aeródromos y aeropuertos.

Se trata de un reporte breve en forma de código alfanumérico que aporta información meteorológica detallada de un momento determinado en un aeropuerto concreto. Básicamente, es una sucesión de letras y números que se emiten periódicamente por los aeropuertos y aeródromos. 

Estos partes se emiten periódicamente, generalmente cada 30 minutos, salvo circunstancias excepcionales en las que se pueden emitir partes adicionales. 

Aunque pueden ser dificíl de descifrar para el usuario general, contienen información de gran relevancia para la actividad aérea relativa a la temperatura, precipitaciones, visibilidad en pista, viento y ráfagas entre otro. Por suerte, se pueden encontrar webs que traducen esta información a un lenguaje más amable y que además mantienen registros de los partes en el tiempo.

## Para ello se va a utilizar la técnica de extracción de datos "webscrapping" sobre la web [tutiempo](https://www.tutiempo.net/registros/lemd) que mantiene registros de estos partes desde hace vários años.


### Las librerías que se van a emplear principalmente para el proceso son:
- **selenium** 
- **joblib**
- **pandas** 


In [ ]:
# %pip install joblib

In [1]:
from joblib import Parallel, delayed
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select   # seleccion de un dropdown
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time

from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

from joblib._parallel_backends import LokyBackend
import asyncio

from tqdm.notebook import tqdm
from datetime import datetime, timedelta

## Se habilitan algunas opciones de interés para el driver

In [2]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
opciones.add_argument('--incognito')

# Despues de hacer un estudio sobre la estructura de la web para plantear el proceso de extracción, se observa que en cada página se encuentran exclusivamente los partes de un día en concreto( de media unos 48).

## Además, se comprueba que al pasar de un día a otro el *link* de la web varía siguiendo un patrón claro como el siguiente:

### - ...tutiempo.net/records/lemd/{<span style="color:red">dia</span>}-{<span style="color:red">mes</span>}-{<span style="color:red">año</span>}.html

### Este formate de *links* es de especial ayuda, ya que nos va a permitir paralelizar el proceso con **joblib**, reduciendo considerablemente el tiempo de extracción.

## Vamos, por tanto, a generar los diferentes *links* necesarios, en este caso únicamente los del año 2023, de Enero a Octubre.

## De momento no nos vamos a preocupar de los días de cada mes, generando 31 links para cada uno de los meses

In [231]:
# Función para formatear el día eliminando el cero delante de los números del 1 al 9
def formatear_dia(dia):
    return str(dia) if dia >= 10 else f"{dia}"

# Definir la fecha de inicio y fin
fecha_inicio = datetime(2017, 11, 1)
fecha_fin = datetime(2022, 12, 31)

# Lista para almacenar los enlaces generados
enlaces = []

# Bucle para generar los enlaces para cada día en el rango especificado
while fecha_inicio <= fecha_fin:
    # Formatear la fecha según el formato deseado
    formato_fecha = f"{formatear_dia(fecha_inicio.day)}-{fecha_inicio.strftime('%B')}-{fecha_inicio.year}".lower()
    
    # Crear el enlace y añadirlo a la lista
    enlace = f"https://en.tutiempo.net/records/lemd/{formato_fecha}.html"
    enlaces.append(enlace)
    
    # Avanzar al siguiente día
    fecha_inicio += timedelta(days=1)


In [232]:
len(enlaces)

1887

In [ ]:
# meses = ["january","february","march","april","may","june","july","august","september","october"]
# dias = []
# for i in range(1,32):
#     dias.append(str(i))


In [ ]:
# urls = []
# for mes in meses:
#     for dia in dias:
#         urls.append(f'https://en.tutiempo.net/records/lemd/{dia}-{mes}-{año}.html')

# La siguiente función <span style="color:red">extraer</span>, engloba el proceso de extraccíon de los datos necesarios para cada una de las *urls*.

## Los comentaríos de la función explican paso por paso la lógica del proceso.
## A modo de resumen, el driver entra en la página, *clicka* aceptar en los popups de cookies, extrae los datos necesarios almacenadolos en una tabla y finálmente devuelve un DataFrame con los registro METAR de ese día.

In [ ]:
def extraer(url):
    table = []  # Inicializa table como una lista vacía
    columns = [] # Inicializa columns como una lista vacía
    
    try:
        # inicia el driver en la url indicada
        driver = webdriver.Chrome()
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        
        time.sleep(3) #espera a cargar la página
        
        # acepta normas
        aceptar = driver.find_element(By.XPATH,'/html/body/div[18]/div[2]/div[1]/div[2]/div[2]/button[1]')
        aceptar.click()

        time.sleep(3) #espera a cargar la página
        
        #acepta cookies    
        aceptar = driver.find_element(By.XPATH, '//*[@id="DivAceptarCookies"]/div/a[2]')
        aceptar.click()
        
        time.sleep(4) #espera a cargar la página
        
        day = driver.find_elements(By.XPATH, '//table//tbody//tr')[1].text

        table = [row.text.split('\n')[0:3] + row.text.replace(' km/h', '').split('\n')[-1].split(' ', 2)
                 for row in driver.find_elements(By.XPATH, '//table//tbody//tr')[3::2]]  #copia los registros
     
        columns = ["Day", "Hour", "Condition", "Temperature", "Wind", "Relative_hum", "Pressure"] #añade las columnas

        for i in table:
            i.insert(0, day) #inserta los registros en la tabla
            
        return pd.DataFrame(table, columns=columns) #devuelve un dataframe con los datos de ese día 
        
    except Exception as e:  # Captura cualquier excepción y muestra el mensaje de error
        print(f"Error: {e}")
        print(f"Error en la URL: {url}")
        # En caso de error, simplemente se devolverá la lista vacía 'table'
        return pd.DataFrame(table, columns=columns) 
    driver.quit()
    return pd.DataFrame(table, columns=columns)

## El siguiente código inicia el proceso de extracción usando la paralelización con 8 núcleos, es decir, realiza el proceso en *8 urls* simultáneamente.

Aunque se podría haber utilizado el proceso una única vez, se decide hacer una extracción por cada uno de los meses, mediante los índices de la tabla de urls, con el objetivo de tener un mayor control ante posibles incidencias del proceso, y poder obtener diferentes registros mensuales.

In [ ]:
dias = []

In [59]:
def extraer(url):
    table = []  # Inicializa table como una lista vacía
    columns = []  # Inicializa columns como una lista vacía
    
    try:
        # Inicia el driver en la URL indicada
        driver = webdriver.Chrome()
        driver.get(url)
        wait = WebDriverWait(driver, 60)
        
        # Esperar a que aparezca el botón de aceptar normas
        normas_button_xpath = '/html/body/div[18]/div[2]/div[1]/div[2]/div[2]/button[1]'
        wait.until(EC.element_to_be_clickable((By.XPATH, normas_button_xpath)))
        normas_button = driver.find_element(By.XPATH, normas_button_xpath)
        normas_button.click()
        print("Aceptado normas")

        # Esperar a que aparezca el botón de aceptar cookies
        cookies_button_xpath = '//*[@id="DivAceptarCookies"]/div/a[2]'
        wait.until(EC.element_to_be_clickable((By.XPATH, cookies_button_xpath)))
        cookies_button = driver.find_element(By.XPATH, cookies_button_xpath)
        cookies_button.click()
        print("Aceptadas cookies")
        
        # Esperar a que cargue la página (usando un elemento en la tabla como referencia)
        wait.until(EC.presence_of_element_located((By.XPATH, '//table//tbody//tr[3]')))
        print("Página cargada")

        day = driver.find_elements(By.XPATH, '//table//tbody//tr')[1].text

        time.sleep(5)
        table = [
            row.text.split('\n')[0:3] + row.text.replace(' km/h', '').split('\n')[-1].split(' ', 2)
            for row in driver.find_elements(By.XPATH, '//table//tbody//tr')[3::2]
        ]  # Copia los registros
        print("Registros extraídos")
        
        columns = ["Day", "Hour", "Condition", "Temperature", "Wind", "Relative_hum", "Pressure"]  # Añade las columnas

        for i in table:
            i.insert(0, day)  # Inserta los registros en la tabla
            
        return pd.DataFrame(table, columns=columns)  # Devuelve un DataFrame con los datos de ese día
        
    except TimeoutException as te:  # Captura TimeoutException
        print(f"Excepción de tiempo de espera: {te}")
        print(f"No se pudo completar la extracción en la URL: {url}")
        return pd.DataFrame(table, columns=columns) 
    except Exception as e:  # Captura otras excepciones
        print(f"Error: {e}")
        print(f"Error en la URL: {url}")
        return pd.DataFrame(table, columns=columns) 
    finally:
        driver.quit()

In [256]:
%%time
paralelo = Parallel(n_jobs=1, verbose=True)

lst_df = paralelo(delayed(extraer)(url) for url in tqdm(enlaces2))

  0%|          | 0/3 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Aceptado normas
Aceptadas cookies
Página cargada
Registros extraídos
Aceptado normas
Aceptadas cookies
Error: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.199)
Stacktrace:
	GetHandleVerifier [0x00007FF62DEF82B2+55298]
	(No symbol) [0x00007FF62DE65E02]
	(No symbol) [0x00007FF62DD205AB]
	(No symbol) [0x00007FF62DD00038]
	(No symbol) [0x00007FF62DD86BC7]
	(No symbol) [0x00007FF62DD9A15F]
	(No symbol) [0x00007FF62DD81E83]
	(No symbol) [0x00007FF62DD5670A]
	(No symbol) [0x00007FF62DD57964]
	GetHandleVerifier [0x00007FF62E270AAB+3694587]
	GetHandleVerifier [0x00007FF62E2C728E+4048862]
	GetHandleVerifier [0x00007FF62E2BF173+4015811]
	GetHandleVerifier [0x00007FF62DF947D6+695590]
	(No symbol) [0x00007FF62DE70CE8]
	(No symbol) [0x00007FF62DE6CF34]
	(No symbol) [0x00007FF62DE6D062]
	(No symbol) [0x00007FF62DE5D3A3]
	BaseThreadInitThunk [0x00007FFF9CA3257D+29]
	RtlUserThreadStart [0x00007FFF9D6CAA58+40]

Error en l

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.8min finished


## Obtenidos los DataFrame para cada unos de los días del mes, se contatenan los DataFrame y se hace una breve exploración sobre el resultado para observar si se han obtenido todos los registros necesarios.

In [14]:

pd.set_option('display.max_rows', None)
metar_2018 = pd.concat(lst_df)
print(len(metar_2018))

16840


In [15]:
len(metar_2018.Day.value_counts())

347

In [39]:
# metar_2018.Day.value_counts()

In [32]:
lista2018 = list(metar_2018.Day.unique())
# lista2018

In [22]:
metar_2019 = pd.concat(lst_df)
print(len(metar_2019))

16826


In [23]:
len(metar_2019.Day.value_counts())

347

In [29]:
# metar_2019[metar_2019.Day == 'Thursday 31 January 2019']

In [40]:
# metar_2019.Day.value_counts()

In [33]:
lista2019 = list(metar_2019.Day.unique())

In [41]:
metar_2020 = pd.concat(lst_df)
print(len(metar_2020))

17455


In [42]:
len(metar_2020.Day.value_counts())

361

In [51]:
# metar_2020.Day.value_counts()

In [44]:
metar_2020.Day.value_counts()

In [53]:
metar_2021 = pd.concat(lst_df)
print(len(metar_2021))

17451


In [54]:
len(metar_2021.Day.value_counts())

359

In [61]:
# metar_2021.Day.value_counts()

In [62]:
metar_2022 = pd.concat(lst_df)
print(len(metar_2022))

17432


In [63]:
len(metar_2022.Day.value_counts())

359

In [64]:
metar_2022.Day.value_counts()

Saturday 26 March 2022         98
Monday 19 December 2022        55
Thursday 3 March 2022          55
Tuesday 26 April 2022          54
Tuesday 20 December 2022       54
Tuesday 13 December 2022       53
Thursday 8 December 2022       53
Monday 5 December 2022         53
Saturday 12 March 2022         53
Monday 10 October 2022         53
Sunday 25 December 2022        53
Wednesday 14 December 2022     53
Friday 22 April 2022           52
Monday 21 November 2022        52
Friday 4 November 2022         52
Friday 21 October 2022         52
Tuesday 15 November 2022       52
Tuesday 12 April 2022          52
Monday 12 December 2022        52
Saturday 18 June 2022          52
Tuesday 20 September 2022      52
Wednesday 21 December 2022     52
Thursday 15 December 2022      51
Sunday 23 October 2022         51
Friday 25 March 2022           51
Monday 21 March 2022           51
Wednesday 3 August 2022        51
Monday 14 March 2022           51
Tuesday 13 September 2022      51
Sunday 18 Dece

In [70]:
metar_2018.head()

,Day,Hour,Condition,Temperature,Wind,Relative_hum,Pressure
0,Monday 1 January 2018,00:00,Clear,8°,19,57%,1030 hPa
1,Monday 1 January 2018,00:30,Clear,7°,15,66%,1030 hPa
2,Monday 1 January 2018,01:00,Clear,7°,20,66%,1030 hPa
3,Monday 1 January 2018,01:30,Clear,8°,19,61%,1030 hPa
4,Monday 1 January 2018,02:00,Clear,8°,20,57%,1030 hPa


In [52]:
lista2020 = list(metar_2020.Day.unique())

In [56]:
lista2021 = list(metar_2021.Day.unique())

In [65]:
lista2022 = list(metar_2022.Day.unique())

## Finalmente se exportan a un archivo *.csv* cada uno de los DataFrames mensuales

In [18]:
metar_2018.to_csv("../data/metars/metars_2018_347.csv", index=False)

In [34]:
metar_2019.to_csv("../data/metars/metars_2019_347.csv", index=False)

In [45]:
metar_2020.to_csv("../data/metars/metars_2020_361.csv", index=False)

In [57]:
metar_2021.to_csv("../data/metars/metars_2021_359.csv", index=False)

In [66]:
metar_2022.to_csv("../data/metars/metars_2022_359.csv", index=False)

In [71]:
metar_2017 = pd.read_csv("../data/metars/metars_2017.csv")

In [137]:
metar_2018 = pd.read_csv("../data/metars/metars_2018_347.csv")

In [169]:
metar_2019 = pd.read_csv("../data/metars/metars_2019_347.csv")

# Concat

In [217]:
metars_18_22 = pd.concat([metar_2017,metar_2018,metar_2019,metar_2020,metar_2021,metar_2022])

In [221]:
metars_18_22=metars_18_22.drop_duplicates()

In [223]:
metars_18_22 = metars_18_22[metars_18_22.Pressure.notna()]

In [224]:
metars_18_22 = metars_18_22.reset_index(drop = True)

In [230]:
len(list(metars_18_22.Day.unique()))

1839

In [264]:
# Convertir la lista existente a objetos datetime
dias_existente = list(metars_18_22.Day.unique())
dias_existente = [datetime.strptime(dia, '%A %d %B %Y') for dia in dias_existente]

# Rango de fechas deseado
fecha_inicio = datetime(2017, 11, 1)
fecha_fin = datetime(2022, 12, 31)

# Generar lista completa de días entre las fechas
dias_completo = [fecha_inicio + timedelta(days=d) for d in range((fecha_fin - fecha_inicio).days + 1)]

# Filtrar los días que no están en la lista existente
dias_faltantes = [dia.strftime('%A %d %B %Y') for dia in dias_completo if dia not in dias_existente]
len(dias_faltantes)

1

In [265]:
# Convertir la lista de días al formato deseado
dias_formato_fecha = [datetime.strptime(dia, '%A %d %B %Y').strftime('%d-%B-%Y').lower() for dia in dias_faltantes]

# Eliminar el 0 delante de los días del 1 al 9
dias_formato_fecha = [fecha[1:] if fecha.startswith('0') else fecha for fecha in dias_formato_fecha]

# Generar enlaces
enlaces2 = [f"https://en.tutiempo.net/records/lemd/{formato_fecha}.html" for formato_fecha in dias_formato_fecha]
enlaces2

['https://en.tutiempo.net/records/lemd/14-october-2020.html']

In [257]:
metar_res2 = pd.concat(lst_df)
print(len(metar_res2))

96


In [258]:
len(metar_res2.Day.value_counts())

2

In [248]:
# metar_res1.Day.value_counts()

In [259]:
metar_res2 = metar_res2.drop_duplicates()

In [260]:
metar_res2 = metar_res2[metar_res1.Pressure.notna()]

In [261]:
metar_res2 = metar_res2.reset_index(drop = True)

In [263]:
metars_18_22 = pd.concat([metars_18_22,metar_res2])

# 2022

In [205]:
metar_2022 = metar_2022.drop_duplicates()

In [206]:
metar_2022 = metar_2022[metar_2022.Pressure.notna()]

In [207]:
metar_2022 = metar_2022.reset_index(drop = True)

In [216]:
# metar_2022.Day.value_counts()

In [212]:
metar_2022.Day.loc[4057:4100] = "Sunday 27 March 2022"

In [210]:
metar_2022.drop([4101,4102], inplace=True)

In [214]:
# metar_2022[metar_2022.Day == "Sunday 27 March 2022"]

# 2021

In [186]:
metar_2021 = metar_2021.drop_duplicates()

In [187]:
metar_2021 = metar_2021[metar_2021.Pressure.notna()]

In [188]:
metar_2021 = metar_2021.reset_index(drop = True)

In [204]:
# metar_2021.Day.value_counts()

In [191]:
metar_2021.drop([4145,4146], inplace=True)

In [199]:
metar_2021.Day.loc[4101:4144] = "Sunday 28 March 2021"

In [202]:
# metar_2021[metar_2021.Day == "Sunday 28 March 2021"]

# 2020

In [155]:
metar_2020 = metar_2020.drop_duplicates()

In [156]:
metar_2020 = metar_2020[metar_2020.Pressure.notna()]

In [157]:
metar_2020 = metar_2020.reset_index(drop = True)

In [196]:
# metar_2020.Day.value_counts()

In [168]:
# metar_2020[metar_2020.Day == "Sunday 29 March 2020"]

In [160]:
metar_2020.drop([4266,4267], inplace=True)

In [163]:
metar_2020.Day.loc[4222:4265] = "Sunday 29 March 2020"

# 2019

In [170]:
metar_2019 = metar_2019.drop_duplicates()

In [171]:
metar_2019 = metar_2019[metar_2019.Pressure.notna()]

In [181]:
metar_2019 = metar_2019.reset_index(drop = True)

In [180]:
metar_2019.Day.loc[4173:4218] = "Monday 1 April 2019"

In [193]:
# metar_2019[metar_2019.Day == "Monday 1 April 2019"]

In [194]:
# metar_2019.Day.value_counts()

# 2018

In [138]:
metar_2018 = metar_2018.drop_duplicates()

In [147]:
metar_2018 = metar_2018.reset_index(drop = True)

In [146]:
metar_2018.Day.loc[3947:3990] = "Sunday 25 March 2018"

In [144]:
metar_2018.drop([3991,3992], inplace=True)

In [195]:
# metar_2018.Day.value_counts()

In [139]:
metar_2018 = metar_2018[metar_2018.Pressure.notna()]

In [154]:
metar_2019.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16820 entries, 0 to 16819
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Day           16820 non-null  object
 1   Hour          16820 non-null  object
 2   Condition     16820 non-null  object
 3   Temperature   16820 non-null  object
 4   Wind          16820 non-null  object
 5   Relative_hum  16820 non-null  object
 6   Pressure      16820 non-null  object
dtypes: object(7)
memory usage: 7.6 MB


# Final

In [269]:
metars_18_22.shape

(91168, 7)

In [271]:
metars_18_22.to_csv("../data/metars/metars_2017_2022.csv", index=False)